In [ ]:
"""
* login to https://ui.hpccloud.surfsara.nl/
* make sure you are in the user view
* go to Apps
* select an "app" (like Ubuntu-16.04.3-Server (2017-12-07)) and press "openNebula"
* select local_images_ssh datastore and then press "download"
* Go to VM Templates view and notice VM template ID
"""

In [ ]:
import oca
import os
import sys

In [ ]:
# settings
TEMPLATE_ID = 7890
MEMORY = 2048 # GB
NODES_NUM = 10

In [ ]:
# things you probably should not change
config_path = os.path.expanduser("~/.surfsara")
endpoint = 'https://api.hpccloud.surfsara.nl/RPC2'

In [ ]:
if not os.access(config_path, os.R_OK):
    print("can't read {}".format(config))
    print(__doc__)
    sys.exit(1)

In [ ]:
config = {}
with open(config_path, 'r') as f:
    for l in f:
        s = l.strip().split('=')
        if len(s) == 2:
            config[s[0]] = s[1]

In [ ]:
if not ('user' in config and 'pass' in config):
    print("can't find user and/or pass in config")
    print(__doc__)
    sys.exit(1)

In [ ]:
client = oca.Client(config['user'] + ':' + config['pass'], endpoint)

In [ ]:
# print a list of nodes
vp = oca.VirtualMachinePool(client)
vp.info()
print("[nodes]")
for vm in vp:
    if vm.state != vm.ACTIVE:
        continue
    ip_list = list(v.ip for v in vm.template.nics)
    print(ip_list[0])

In [ ]:
extra_template = """
MEMORY = {}
""".format(MEMORY)

In [ ]:
# instanciate one VM
tp = oca.VmTemplatePool(client)
tp.info()
template = tp.get_by_id(TEMPLATE_ID)
template.instantiate(extra_template=extra_template)

In [ ]:
# instantiate multiple
for i in range(NODES_NUM):
    template.instantiate(name="workflow-{}".format(i))

In [ ]:
# delete all VM's
vmp = oca.VirtualMachinePool(client=client)
vmp.info()

for i in vmp:
    i.delete()